In [0]:
!pip install tensorflow-gpu==2.0.0-beta0
!pip install tensorflow_hub

     |████████████████████████████████| 348.9MB 93kB/s 
     |████████████████████████████████| 3.1MB 33.2MB/s 
     |████████████████████████████████| 501kB 49.0MB/s 


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from google.colab import files

In [0]:
import tensorflow as tf
from keras.models import Model
model = "/content/gdrive/My Drive/pdd_classifier_all_150epochs_100epochs.h5"
SAVED_MODEL = "/content/gdrive/My Drive/pdd_model_new/pdd/"
tf.keras.experimental.export_saved_model(tf.keras.models.load_model(model), SAVED_MODEL)

Using TensorFlow backend.
W0724 06:35:44.693661 140707236595584 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0724 06:35:46.838095 140707236595584 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:253: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
W0724 06:35:46.839488 140707236595584 export_utils.py:182] Export includes no default signature!
W0724 06:35:47.862777 

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [0]:
# Load SavedModel

model = tf.keras.experimental.load_from_saved_model(SAVED_MODEL)

W0724 06:35:53.252265 140707236595584 util.py:244] Unresolved object in checkpoint: (root).layer_with_weights-1._updates
W0724 06:35:53.253264 140707236595584 util.py:244] Unresolved object in checkpoint: (root).layer_with_weights-3._updates
W0724 06:35:53.255158 140707236595584 util.py:244] Unresolved object in checkpoint: (root).layer_with_weights-5._updates
W0724 06:35:53.256058 140707236595584 util.py:244] Unresolved object in checkpoint: (root).layer_with_weights-7._updates
W0724 06:35:53.257090 140707236595584 util.py:244] Unresolved object in checkpoint: (root).layer_with_weights-9._updates
W0724 06:35:53.258044 140707236595584 util.py:244] Unresolved object in checkpoint: (root).optimizer.decay
W0724 06:35:53.258981 140707236595584 util.py:244] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0724 06:35:53.259969 140707236595584 util.py:244] Unresolved object in checkpoint: (root).optimizer.momentum
W0724 06:35:53.261082 140707236595584 util.py:244] Unresolved o

In [0]:
!mkdir "/content/gdrive/My Drive/pdd_model_new/tfmodels/"

mkdir: cannot create directory ‘/content/gdrive/My Drive/pdd_model_new/tfmodels/’: File exists


In [0]:
#TFLITE_MODEL = "/content/gdrive/My Drive/pdd_model_new/tfmodels/pdd.tflite"
TFLITE_MODEL = "/content/gdrive/My Drive/pdd_model_new/tfmodels/model.tflite"
TFLITE_QUANT_MODEL = "/content/gdrive/My Drive/pdd_model_new/tfmodels/model_quant.tflite"

In [0]:
# Get the concrete function from the Keras model.
run_model = tf.function(lambda x : model(x))

# Save the concrete function.
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype)
)

# Convert the model
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converted_tflite_model = converter.convert()
open(TFLITE_MODEL, "wb").write(converted_tflite_model)

# Convert the model to quantized version with post-training quantization
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_quant_model = converter.convert()
open(TFLITE_QUANT_MODEL, "wb").write(tflite_quant_model)

print("TFLite models and their sizes:")
!ls "/content/gdrive/My Drive/pdd_model_new/tfmodels/" -lh

TFLite models and their sizes:
total 214M
-rw------- 1 root root  15M Jul 24 06:38 model_quant.tflite
-rw------- 1 root root  57M Jul 24 06:38 model.tflite
-rw------- 1 root root  57M Jul 23 08:08 pdd.lite
-rw------- 1 root root  57M Jul 23 14:42 pdd_noexp.lite
-rw------- 1 root root 9.3K Jul 23 16:03 pdd_noexp.npz
-rw------- 1 root root 9.3K Jul 23 08:20 pdd.npz
-rw------- 1 root root  15M Jul 23 08:08 pdd_quant.lite
-rw------- 1 root root  15M Jul 23 14:42 pdd_quant_noexp.lite
-rw------- 1 root root 9.3K Jul 23 15:55 pdd_quant_noexp.npz
-rw------- 1 root root 9.3K Jul 23 09:44 pdd_quant.npz
